In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

In [3]:
class SNN(nn.Module):
    def __init__(self):
        super(SNN, self).__init__()
        self.first_layer = nn.Linear(28*28,32)
        self.second_layer = nn.Linear(32,10)
        
    def forward(self,x):
        y1 = F.relu(self.first_layer(x))
        y = self.second_layer(y1)
        return y

In [4]:
transform_train = transforms.Compose([transforms.ToTensor()])
transform_test = transforms.Compose([transforms.ToTensor()])

In [5]:
mnist_train = torchvision.datasets.MNIST(root='./MNIST',train=True,download=True,transform=transform_train)
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=100,shuffle=True)

mnist_test = torchvision.datasets.MNIST(root='./MNIST',train=False,download=True,transform=transform_test)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=100,shuffle=False)

In [6]:
network = SNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(network.parameters(),lr=1e-3)

In [7]:
for i in range(100):
    network.train()
    correct_rate = 0
    for (inputs, targets) in train_loader:
        inputs = inputs.view(inputs.shape[0], -1)
        optimizer.zero_grad()
        outputs = network(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        value, pred = outputs.max(1)
        correct_rate += pred.eq(targets).sum().item()
    print("Epoch "+ str(i) + ": " + str(correct_rate/600) + "% accurate")

Epoch 0: 19.745% accurate
Epoch 1: 34.285% accurate
Epoch 2: 43.52166666666667% accurate
Epoch 3: 53.693333333333335% accurate
Epoch 4: 62.455% accurate
Epoch 5: 68.405% accurate
Epoch 6: 72.65666666666667% accurate
Epoch 7: 75.69333333333333% accurate
Epoch 8: 77.97333333333333% accurate
Epoch 9: 79.61333333333333% accurate


In [9]:
network.eval()
correct_rate = 0
with torch.no_grad():
    for (inputs, targets) in test_loader:
        inputs = inputs.view(inputs.shape[0], -1)
        optimizer.zero_grad()
        outputs = network(inputs)
        values, pred = outputs.max(1)
        correct_rate += pred.eq(targets).sum().item()
print(correct_rate/100)

80.92
